In [9]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import warnings
import argparse
import pickle
import cv2
import os
import sys
import glob
import matplotlib
matplotlib.use("Agg")

In [10]:
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [11]:
args = {
    "dataset": "./SCVD/3x2",
    "model": "./Mobilenet3x2/models/mobilenet_3x2.h5",
    "label-bin": "./Mobilenet3x2/label-bin/mobilenet_3x2.pickle",
    "epochs": 30,
    "plot": "./Mobilenet3x2/plots/mobilenet_3x2.png" 
}

In [12]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["Violence", "NonViolence", "WeaponViolence"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print('-'*100)
print("[INFO] loading images...")
print('-'*100)
from imutils import paths
imagePaths = list(paths.list_images(args["dataset"]))
print(imagePaths)
data = []
labels = []

----------------------------------------------------------------------------------------------------
[INFO] loading images...
----------------------------------------------------------------------------------------------------
['./SCVD/3x2\\NonViolence\\nv100_merge_0.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_1.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_10.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_11.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_12.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_13.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_14.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_15.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_16.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_17.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_18.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_19.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_2.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_20.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_21.jpg', './SCVD/3x2\\NonViolence\\nv100_merge_22.jpg', './SCVD/3x2\\NonViolenc

In [13]:
# loop over the image paths
for imagePath in tqdm(imagePaths[::]):
    # imagePath : file name ex) V_123
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2] # Violence / NonViolence

    # if the label of the current image is not part of of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue

    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

100%|██████████████████████████████████████████████████████████████████████████| 14513/14513 [1:36:08<00:00,  2.52it/s]


In [14]:
# convert the data and labels to NumPy arrays
dt = np.array(data)
lbls = np.array(labels)


# perform one-hot encoding on the labels
lb = LabelBinarizer()
lbls_fit = lb.fit_transform(lbls)
#lbls_cat = to_categorical(lbls_fit, num_classes=3)

# partition the data into training and testing splits using 70% of
# the data for training and the remaining 30% for testing
(trainX, testX, trainY, testY) = train_test_split(dt, lbls_fit, test_size=0.2, stratify=lbls_fit, random_state=33)

In [15]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    #rotation_range=30,
    zoom_range=0.15,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [16]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [17]:
# load the InceptionV3 network, ensuring the head FC layer sets are left
# off
model = MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=3,
    classifier_activation="softmax")

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
model.trainable = True

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print('-'*100)
print("[INFO] compiling model...")
print('-'*100)
opt = SGD(lr=1e-3, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(model.summary())

----------------------------------------------------------------------------------------------------
[INFO] compiling model...
----------------------------------------------------------------------------------------------------
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                 

C:\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


 expanded_conv_depthwise_relu (  (None, 112, 112, 32  0          ['expanded_conv_depthwise_BN[0][0
 ReLU)                          )                                ]']                              
                                                                                                  
 expanded_conv_project (Conv2D)  (None, 112, 112, 16  512        ['expanded_conv_depthwise_relu[0]
                                )                                [0]']                            
                                                                                                  
 expanded_conv_project_BN (Batc  (None, 112, 112, 16  64         ['expanded_conv_project[0][0]']  
 hNormalization)                )                                                                 
                                                                                                  
 block_1_expand (Conv2D)        (None, 112, 112, 96  1536        ['expanded_conv_project_BN[0][0]'
          

                                                                                                  
 block_4_expand_relu (ReLU)     (None, 28, 28, 192)  0           ['block_4_expand_BN[0][0]']      
                                                                                                  
 block_4_depthwise (DepthwiseCo  (None, 28, 28, 192)  1728       ['block_4_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_4_depthwise_BN (BatchNor  (None, 28, 28, 192)  768        ['block_4_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_4_depthwise_relu (ReLU)  (None, 28, 28, 192)  0           ['block_4_depthwise_BN[0][0]']   
          

                                                                                                  
 block_7_add (Add)              (None, 14, 14, 64)   0           ['block_6_project_BN[0][0]',     
                                                                  'block_7_project_BN[0][0]']     
                                                                                                  
 block_8_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_7_add[0][0]']            
                                                                                                  
 block_8_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_8_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_8_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_8_expand_BN[0][0]']      
          

 block_11_depthwise_relu (ReLU)  (None, 14, 14, 576)  0          ['block_11_depthwise_BN[0][0]']  
                                                                                                  
 block_11_project (Conv2D)      (None, 14, 14, 96)   55296       ['block_11_depthwise_relu[0][0]']
                                                                                                  
 block_11_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_11_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_add (Add)             (None, 14, 14, 96)   0           ['block_10_project_BN[0][0]',    
                                                                  'block_11_project_BN[0][0]']    
                                                                                                  
 block_12_

 block_15_expand_relu (ReLU)    (None, 7, 7, 960)    0           ['block_15_expand_BN[0][0]']     
                                                                                                  
 block_15_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_15_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_15_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_15_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_15_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_15_depthwise_BN[0][0]']  
                                                                                                  
 block_15_

In [18]:
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print('-'*100)
print("[INFO] training head...")
print('-'*100)
H = model.fit(trainAug.flow(trainX, trainY, batch_size=32),
            steps_per_epoch=len(trainX) // 32,
            validation_data=valAug.flow(testX, testY),
            validation_steps=len(testX) // 32,
            epochs=args["epochs"])

----------------------------------------------------------------------------------------------------
[INFO] training head...
----------------------------------------------------------------------------------------------------
Epoch 1/30
362/362 [==============================] - 4075s 11s/step - loss: 1.0038 - accuracy: 0.4764 - val_loss: 1.1004 - val_accuracy: 0.3288
Epoch 2/30
362/362 [==============================] - 4239s 12s/step - loss: 0.6865 - accuracy: 0.6939 - val_loss: 1.0990 - val_accuracy: 0.3316
Epoch 3/30
362/362 [==============================] - 2412s 7s/step - loss: 0.4196 - accuracy: 0.8325 - val_loss: 1.0962 - val_accuracy: 0.3625
Epoch 4/30
362/362 [==============================] - 2365s 7s/step - loss: 0.2651 - accuracy: 0.8985 - val_loss: 1.0964 - val_accuracy: 0.3639
Epoch 5/30
362/362 [==============================] - 2424s 7s/step - loss: 0.1882 - accuracy: 0.9317 - val_loss: 1.1094 - val_accuracy: 0.3316
Epoch 6/30
362/362 [==============================] 

In [19]:
# evaluate the network
print('-'*100)
print("[INFO] evaluating network...")
print('-'*100)
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

----------------------------------------------------------------------------------------------------
[INFO] evaluating network...
----------------------------------------------------------------------------------------------------
91/91 [==============================] - 119s 1s/step
                precision    recall  f1-score   support

   NonViolence       1.00      1.00      1.00       962
      Violence       1.00      1.00      1.00      1053
WeaponViolence       0.99      1.00      1.00       888

      accuracy                           1.00      2903
     macro avg       1.00      1.00      1.00      2903
  weighted avg       1.00      1.00      1.00      2903



In [20]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

----------------------------------------------------------------------------------------------------
[INFO] plot the training loss and accuracy...
----------------------------------------------------------------------------------------------------


In [21]:
# serialize the model to disk
print('-'*100)
print("[INFO] serializing network...")
print('-'*100)
model.save(args["model"])

----------------------------------------------------------------------------------------------------
[INFO] serializing network...
----------------------------------------------------------------------------------------------------


In [22]:
# serialize the label binarizer to disk
f = open(args["label-bin"], "wb")
f.write(pickle.dumps(lb))
f.close()